In [2]:
import numpy as np

# 读取txt文件中的数据
data = np.loadtxt('ZnS.txt')  # 确保文件在当前工作目录或提供完整路径[1,2,3](@ref)

# 提取各列数据
wavelength_um = data[:, 0]  # 第一列：波长（微米）
n_real = data[:, 1]         # 第二列：折射率实部
n_imag = data[:, 2]        # 第三列：折射率虚部

# 计算复折射率 (n + ik)
refractive_index_complex = n_real + 1j * n_imag  # 使用Python内置复数表示[8](@ref)

# 计算复相对介电常数 (ε = ñ² = (n + ik)² = n² - k² + i2nk)
epsilon_complex = refractive_index_complex**2

# 将波长转换为频率
# c = λf => f = c/λ
c = 3e8  # 光速（米/秒）
wavelength_m = wavelength_um * 1e-6  # 将微米转换为米
f = c / wavelength_m  # 计算频率（Hz）

# 创建最终的数据结构
sampledData = [f, epsilon_complex]

# 打印部分结果以供验证
print("频率数组（前5个值）:")
print(f[:5])
print("\n复相对介电常数（前5个值）:")
print(epsilon_complex[:5])

# 如果需要保存处理后的数据
# np.save('processed_data.npy', sampledData)

频率数组（前5个值）:
[1.36363636e+15 1.25000000e+15 1.15384615e+15 1.07142857e+15
 1.00000000e+15]

复相对介电常数（前5个值）:
[7.365165+7.952812j 8.718336+4.50512j  8.165496+3.03863j
 7.755967+2.348544j 7.567627+1.980636j]


In [3]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import importlib
import math
import random


#lumapi = importlib.machinery.SourceFileLoader('lumapi', 'E:\\Program Files\\Lumerical\\v241\\api\\python\\lumapi.py').load_module()
lumapi = importlib.machinery.SourceFileLoader('lumapi', 'C:\\Program Files\\Lumerical\\v241\\api\\python\\lumapi.py').load_module()
def basis_function(i, t):
    """二次B样条基函数"""
    if i == 1:
        return (1-t)*(1-t)
    if i == 2:
        return 2*t*(1-t)
    if i == 3:
        return t*t
    return 0

class Simulation:
    def __init__(self, control_points, target_phase = 0):
        self.control_points = control_points  # (N,2) 控制点坐标
        self.fdtd = lumapi.FDTD(hide=False)
        self.um = 1e-6
        self.knots = np.array([0,0,0,1,2,3,4,5,6,7,7,7])
        self.edge_indices = [[0,1,2],[2,3,4],[4,5,6],[6,7,0]]
        self.target_phase = target_phase
        self.fdtd.switchtolayout()
        ZnS_material = self.fdtd.addmaterial("Sampled data")
        self.fdtd.setmaterial(ZnS_material,"name","ZnS")
        self.fdtd.setmaterial("ZnS","max coefficients",6)
        self.fdtd.setmaterial("ZnS","sampled data",np.transpose(np.array(sampledData)))
        self.phase = 0
        self.fdtd.save("nurbs_thermal02.fsp")

    def set_contral_points(self,control_points):
        self.control_points = control_points

    def set_target_phase(self,target_phase):
        self.target_phase = target_phase

    def setup_simulation(self):
        """基础仿真设置"""
        self.fdtd.switchtolayout()
        self.fdtd.deleteall()

        # 基底设置
        self.fdtd.addrect(name="Ge_top")
        self.fdtd.set("material","Ge (Germanium) - Palik")
        self.fdtd.set('z min',-0.1e-6)
        self.fdtd.set('z max',0)
        self.fdtd.set('x',0)
        self.fdtd.set('x span',70e-6)
        self.fdtd.set('y',0)
        self.fdtd.set('y span',70e-6)

        # 基底设置
        self.fdtd.addrect(name="ZnS_mid")
        self.fdtd.set("material","ZnS")
        self.fdtd.set('z min',-0.7e-6)
        self.fdtd.set('z max',-0.1e-6)
        self.fdtd.set('x',0)
        self.fdtd.set('x span',70e-6)
        self.fdtd.set('y',0)
        self.fdtd.set('y span',70e-6)

        # 基底设置
        self.fdtd.addrect(name="Ge_bottom")
        self.fdtd.set("material","Ge (Germanium) - Palik")
        self.fdtd.set('z min',-0.8e-6)
        self.fdtd.set('z max',-0.7e-6)
        self.fdtd.set('x',0)
        self.fdtd.set('x span',70e-6)
        self.fdtd.set('y',0)
        self.fdtd.set('y span',70e-6)

        # 基底设置
        self.fdtd.addrect(name="Au_bottom")
        self.fdtd.set("material","Au (Gold) - Palik")
        self.fdtd.set('z min',-0.85e-6)
        self.fdtd.set('z max',-0.8e-6)
        self.fdtd.set('x',0)
        self.fdtd.set('x span',70e-6)
        self.fdtd.set('y',0)
        self.fdtd.set('y span',70e-6)

        self.fdtd.addmesh(name="fine_mesh")
        self.fdtd.set('z min',-1e-6)
        self.fdtd.set('z max',0.2e-6)
        self.fdtd.set('x',0)
        self.fdtd.set('x span',70e-6)
        self.fdtd.set('y',0)
        self.fdtd.set('y span',20e-6)
        self.fdtd.set("dx",0.1e-6)
        self.fdtd.set("dy",0.1e-6)
        self.fdtd.set("dz",0.1e-6)

        self.fdtd.addplane(name="source")
        self.fdtd.set('wavelength start',11.6e-6)
        self.fdtd.set('wavelength stop',13.6e-6)
        self.fdtd.set('direction','Backward')
        self.fdtd.set('polarization angle',0)
        self.fdtd.set('z',3e-6)
        self.fdtd.set('x',0)
        self.fdtd.set('y',0)
        self.fdtd.set('x span',70e-6)
        self.fdtd.set('y span',20e-6)
        frequency_points = int(2e-6/(1e-8)) + 2
        # 添加监视器
        self.fdtd.addpower(
            name="R",
            x=0, y=0, z=4e-6,
            x_span=70e-6, y_span=20e-6, 
            monitor_type="2D Z-normal"
        )
        self.fdtd.setglobalmonitor("frequency points",frequency_points)


        zmax =  5e-6
        zmin = -1e-6

        self.fdtd.addfdtd(
            dimension="3D",
            x=0, y=0, z=(zmax+zmin)/2,
            x_span=(12.6e-6)*5, y_span=12.6e-6, z_span=zmax-zmin
        )

        self.fdtd.set("y min bc","Periodic")

        self.fdtd.set("simulation time",5000e-15)

        self.generate_structure(self.control_points)     



    def generate_structure(self, curves):
        um = 1e-6
        line_width = np.array([1.1*um,1*um,0.8*um,0.7*um,0.6*um,0.5*um])
        period = 12.6 * um  # 阵列周期
        
        # 存储所有曲线的顶点矩阵
        all_vertices = []
        self.set_contral_points(curves)
        
        for idx, points in enumerate(self.control_points):
            points = np.array(points) * um
            
            # 计算曲线段数
            segments = len(points) - 2
            if segments < 1:
                continue  # 至少需要3个点才能形成一段曲线
                
            num_points_per_segment = 100
            total_points = segments * num_points_per_segment
            
            # 存储中心点
            center_points = np.zeros((total_points, 2))
            
            # 计算曲线上的点
            for seg in range(segments):
                for j in range(num_points_per_segment):
                    t = j / (num_points_per_segment - 1)
                    
                    # 使用基函数计算点位置
                    x = basis_function(1, t) * points[seg][0] + \
                        basis_function(2, t) * points[seg+1][0] + \
                        basis_function(3, t) * points[seg+2][0]
                        
                    y = basis_function(1, t) * points[seg][1] + \
                        basis_function(2, t) * points[seg+1][1] + \
                        basis_function(3, t) * points[seg+2][1]
                        
                    index = seg * num_points_per_segment + j
                    center_points[index] = [x, y]
            
            # 计算法线方向
            tangents = np.zeros_like(center_points)
            tangents[1:-1] = center_points[2:] - center_points[:-2]
            tangents[0] = center_points[1] - center_points[0]
            tangents[-1] = center_points[-1] - center_points[-2]
            
            # 归一化切线向量
            norms = np.linalg.norm(tangents, axis=1)
            norms[norms == 0] = 1e-10  # 避免除以零
            tangents /= norms[:, np.newaxis]
            
            # 计算法线向量（旋转90度）
            normals = np.column_stack((-tangents[:, 1], tangents[:, 0]))
            
            # 计算多边形顶点
            top_points = center_points + normals * line_width[idx] / 2
            bottom_points = center_points - normals * line_width[idx] / 2
            
            # 创建多边形顶点矩阵
            V = np.vstack((
                top_points,
                bottom_points[::-1],
                top_points[0:1]  # 闭合多边形
            ))
            
            all_vertices.append(V)
        
        # 创建Lumerical脚本
        script = "um = 1e-6;\n"
        script += "period = 12.6 * um;\n"  # 定义周期
        
        array_size = 5  # 5x5阵列
        
        for i, V in enumerate(all_vertices):
            # 为每个原始结构创建阵列
            for row in range(array_size):
                for col in range(array_size):
                    # 计算偏移量
                    x_offset = (col-2) * period
                    y_offset = (row-2) * period
                    
                    # 应用偏移到所有顶点
                    V_offset = V.copy()
                    V_offset[:, 0] += x_offset  # X方向偏移
                    V_offset[:, 1] += y_offset  # Y方向偏移
                    
                    # 转换为Lumerical矩阵格式
                    vertices_str = "[" + ";\n".join(
                        [f"{x}, {y}" for x, y in V_offset]
                    ) + "]"
                    
                    name = f"nurbs_waveguide_{i}_{row}_{col}"
                    script += f"""
        addpoly;
        set("name", "{name}");
        set("x", 0);
        set("y", 0);
        set("z", 0);
        set("z span", 0.05e-6);
        set("vertices", {vertices_str});
        set("material", "Ag (Silver) - Johnson and Christy");
        """
        
        self.fdtd.eval(script)
    
    
    def run_forward(self, wavelength_start=400e-9,wavelength_stop=700e-9):
        """运行正向仿真"""
        self.fdtd.switchtolayout()
        self.fdtd.run()
        Reflect = (self.fdtd.getresult('R','T'))["T"]
        phase = np.angle(self.fdtd.getdata('phase','Ex'))
        self.phase = phase

        return Reflect,phase


In [7]:
wavelength_start=8e-6
wavelength_stop=12e-6
d = np.array([0.2,0.3,0.5,0.7,1,1])
l = np.array([5,4.8,4.6,3.5,3,3])
points2 = np.array([[[-5.25+d[0],-l[0]],[-5.25,0],[-5.25+d[0],l[0]]],
                    [[-3.15+d[1],-l[1]],[-3.15,0],[-3.15+d[1],l[1]]],
                    [[-1.05+d[2],-l[2]],[-1.05,0],[-1.05+d[2],l[2]]],
                    [[1.05+d[3],-l[3]],[1.05,0],[1.05+d[3],l[3]]],
                    [[3.15+d[4],-l[4]],[3.15,0],[3.15+d[4],l[4]]],
                    [[5.25+d[5],-l[5]],[5.25,0],[5.25+d[5],l[5]]]
                    ])
sim = Simulation(points2)
sim.set_contral_points(points2)
sim.setup_simulation()
#Reflect,phase = sim.run_forward(wavelength_start,wavelength_stop)

In [7]:
wavelength_start=8e-6
wavelength_stop=12e-6

points2 = np.array([[[-4,-6],[-5,0],[-4,6]],[[-5,0],[0,0],[5,0]],[[4,-4],[5,0],[4,4]]])
sim = Simulation(points2)
sim.set_contral_points(points2)
sim.setup_simulation()
Reflect,phase = sim.run_forward(wavelength_start,wavelength_stop)

(201, 2)
(201, 2)
(201, 2)


In [43]:
wavelength_start=8e-6
wavelength_stop=12e-6

points2 = np.array([[[-0,-1],[-0,0],[-0,2]],[[-0,0],[0,1],[1,0]]])
sim = Simulation(points2)
sim.set_contral_points(points2)
sim.setup_simulation()
Reflect,phase = sim.run_forward(wavelength_start,wavelength_stop)

(201, 2)
(201, 2)
